# Optimization for Training Deep Models

 There are many types of optimization problems in deep learning, the major one being training a neural network. The one discussed here in great detail is finding the parameter $\theta$ that can reduce the cost function $J(\theta)$
 
### Difference between learning and pure optimization:

1. Machine Learning acts indirectly, the performance measure P we care about is defined with respect to the test set and is often intractable, so we can only optimize it indirectly using $J(\theta)$. This is in contrast to pure optimization where the goal is to minimize J itself. 
2. The expected generalization error(risk) is taken over the true data generating distribution p_data. If we do have that, it becomes an optimization problem. - $J(\theta)$ = $E_{(x,y)\sim p_{data}}L(f(x;\theta),y)$

#### Empirical Risk Minimization
When we don't have $p_{data}$ but have training data then it becomes an ML problem, which can be converted to optimization problem by converting $p_{data}$ into $\hat{p}_{data}$, thereby reducing the empirical risk. This is called ERM (Empirical Risk Minimization) $E_{(x,y)\sim \hat{p}_{data}}L(f(x;\theta),y) = \frac{1}{m}\Sigma_{1}^{m} L(f(x^{(i)};\theta),y^{(i)})$

#### Surrogate Loss Functions and Early Stopping
When the loss function we actually care about cannot be optimized efficiently, then we use a surrogate loss function instead. For e.g. the 0-1 loss is typically intractable and we use negative log likelihood as a surrogate to that. It can have advantages like in some cases the test-set 0/1 loss continues to decrease even after the training set 0/1 has reached zero, because one can improve the robustness by making the classes further apart if a negative log likelihood function is used. 

#### Batch and Minibatch Algorithms
In optimization we use the whole batch, that is sum over all training examples. In case of ML we usually compute the updates to the parameter using estimate of the cost function on a subset of the terms of the full cost function.

The standard error of the mean estimated from n samples is given by $\frac{\sigma}{\sqrt{n}}$, so compare two cases one if we compute the gradient using 100 samples and other using 10,000 samples then the latter requires 100 times more computation but the former reduces the standard error of mean only by a factor of 10. The algorithms converge much faster if they are allowed to rapidly compute approximate gradients rather than slowly computing exact gradients.

There are three methods:

1. Batch Gradient Descent - Where entire batch is used to make a single update.
2. Stochastic Gradient Descent - Where a single example is used to make weight update. 
3. Mini-batch Gradient Descent - Where a batch of examples is randomly sampled from entire training set to make an update.


Minibatch sizes are driven by the following factors:

1. Larger batch size provide a more accurate estimate of the gradient but less than linear returns.
2. Multicore architectures are usually underutilized by extremely small batches. This motivates using some absolute minimum batch size, below which there is no reduction in the time to process a minibatch
3. If all examples in the batch are processed in parallel, then the amount of memory scales with the batch size. For many hardware setups this is the limiting factor in chosing batch size.
4. Some kinds of hardware achieve better runtime with specific sizes of arrays. Especially when using GPUs, it is common for power of 2 batch sizes to offer better runtime. Typical power of 2 batch sizes range from 32 to 256, with 16 for large models.
5. Small batch sizes can offer regularization effect, perhaps due to the noise they add to the learning process. Generalization error is often the best for batch size of 1.  Training with such a small batch size might require a small learning rate to maintain stability because of the high variance in the estimate of gradient. The total runtime as a result is very high, because it needs more steps as the learning rate is low and it needs more time to see the entire training set.

It is also crucial to select mini batch sizes randomly. It is sufficient to shuffle the dataset once and iterate over it multiple times. In the first epoch the network sees an example for the first time and hence the estimate of the gradient is an unbiased estimate of the true generalization error. However, from the second epoch onwards the estimate becomes biased as it is re-sampling from the data it has already seen. Such a sampling algorithm is called Random Reshuffling and although their analysis even for generalized linear models, which are strongly convex, is an open problem till date, reasonable efforts have been made to show that this biased estimate of the gradient is decent enough (at least as good as i.i.d. based Batch SGD).

### Challenges in Neural Network Optimization
Traditionally ML algorithms have avoided the optimization difficulties by keeping the objective function convex but while training neural networks, we have to confront the non-convex case.
#### Local Minima
In case of convex optimization we could reduce it to the problem of finding a local minima and that local minima was guaranteed to be a global minima as well. Some convex functions have a flat region at the bottom rather than a single global minimum point but any point in that region is acceptable solution. Thus, when optimizing convex functions we know that we have a good solution of any kind of critical point is achieved. With non convex functions we can have many local minima, but we will see it is not necessarily a problem.
<p>
    <img src="local_minima.gif" alt>
    <center><em>There might be many local minima where the model may get stuck due to zero gradient</em></center>
</p>

Nearly any deep learning model is guaranteed to have an extremely large number of local minima due to model identifiability problem. A model is said to be identifiable if a sufficiently large training set can rule out all but one setting of the model parameters. In case of neural networks, we can obtain equivalent models by swapping the position of the neurons. Thus, they are not identifiable.

<p>
    <img src="model_identifiability.png" alt>
    <center><em>Swapping the two hidden nodes leads to equivalent models. Thus, even after having a sufficiently large training set, there is not a unique setting of parameters. This is the model identifiability problem that neural networks suffer from.</em></center>
</p>

It is nowadays suspected that for sufficiently large neural networks, most local minima have a low cost function value, and it is not important to find a true global minima rather than to find a point in parameter space that has low but not minimal cost. A test to rule out local minima as a problem is plotting the norm of gradient over time. If the norm of gradient does not shrink to an insignificant size, the problem is neither local minima nor any kind of critical point.

#### Plateaus, Saddle Points and Other Flat Regions

For high dimensional spaces, local minima are rare compared to another kind of point called saddle point. Saddle point (SP) is another type of point with zero gradient where some points around it have higher value and the others have lower. Intuitively, this means that a saddle point acts as both a local minima for some neighbors and a local maxima for the others. Thus, Hessian at SP has both positive and negative eigenvalues.For a function to curve upwards or downwards around a point as in the case of local minima and local maxima, the eigenvalues should have the same sign, positive for local minima and negative for local maxima).
<p>
    <img src="saddle_local_min.png" alt>
    <center><em>Saddle Point and Local Minima</em></center>
</p>
In early days of deep learning, people used to worry about local minima, they used to draw a picture like in the right, and it looks like there are a lot of local minima in this plot. But, in a very high dimensional space it turns out that most of the points of zero gradient in cost function are not local minima but are rather saddle points. In a function of very high dimensional space, if the gradient is zero, then in each direction it can either be a convex light function or a concave light function. And if you are in, say, a 20,000 dimensional space, then for it to be a local optima, all 20,000 directions need to look like this. And so the chance of that happening is  very small ($2^{-20000}). Instead you're much more likely to get some directions where the curve bends up like so, as well as some directions where the curve function is bending down rather than have them all bend upwards. So that's why in very high-dimensional spaces you're actually much more likely to run into a saddle point like that shown on the right, then the local optimum. A lot of our intuitions about low dimensional spaces really don't transfer to high dimensional spaces.

<p>
    <img src="plateaus.png" alt>
    <center><em>Problem of plateaus</em></center>
</p>

If local optima aren't a problem, then what is a problem? It turns out that plateaus can really slow down learning and a plateau is a region where the derivative is close to zero for a long time. So if you're here, then gradient descents will move down the surface, and because the gradient is zero or near zero, the surface is quite flat. You can actually take a very long time, you know, to slowly find your way to maybe this point on the plateau. And then because of a random perturbation of left or right, maybe then finally I'm going to search pen colors for clarity. Your algorithm can then find its way off the plateau. Let it take this very long slope off before it's found its way here and they could get off this plateau.

#### Cliffs and Exploding Gradients

Neural Networks might sometimes have extremely steep regions resembling cliffs due to repeated multiplication of weights. Suppose we use three layer neural network with all activations as linear. We choose the same number of input, hidden and output neurons, thus, using the same weight W for each layer. The output layer y = W*h where h = W*x represents the hidden layer, finally giving y = W*W x. So, deep neural networks involve multiplication of a large number of parameters leading to sharp non-linearities in the parameter space. These non-linearities give rise to high gradients in some places. At the edge of such a cliff, an update step might throw the parameters extremely far.
<p>
    <img src="cliffs.png" alt>
</p>

#### Long Term dependencies
This problem arises when the computational graph becomes very deep. Consider a comptational graph containing a path that consists of multiplying W repeatedly, after t steps it is equivalent to multipying by $W^t$. Suppose that W has eigen decomposition $W = Vdiag(\lambda)V^{-1}.$ In this simple case we can see that $W^t = Vdiag(\lambda)^tV^{-1}$. Any eigenvalues $\lambda_i$ that are not near absolute value 1 will either explode if greater than 1 or vanish if less than 1. The <b>vanishing and exploding gradient problem</b> refers to the fact that gradietnts through such a graph are scaled according to $diag(\lambda)^t$.

Vanishing gradients make it difficult to know which direction the parameters should move to improve the cost function, while exploding gradients make learning unstable.
<p>
    <img src="vanish_explode.jpeg" alt>
</p>

#### Inexact Gradients

Most optimization algorithms are designed under the assumption that we have access to exact gradient or Hessian Matrix, while in reality we only have a noisy and even a biased estimate of these quantities. In some cases, the objective function we want to minimize is intractable, typically its gradient is intractable as well. In such cases we can only approximate the gradient. One can also avoid this issue by chosing a surrogate loss function that is easier to approximate than the true loss.

### Algorithms

#### Stochastic Gradient Descent:

It is possible to obtain an unbiased estimate of the gradient by takinh the average gradient on a minibatch of m examples drawn i.i.d from the data-generating distribution.

A crucial parameter for the SGD algorithm is the learning rate. It is necessary to decrease the learning rate over time. This is because random sampling of batches act as a source of noise which might make SGD keep oscillating around the minima without actually reaching it. 

<p>
    <img src="SGD.png" alt>
</p>

Setting it too low makes the training proceed slowly which might lead to the algorithm being stuck at a high cost value. Setting it too high would lead to large oscillations which might even push the learning outside the optimal region. The best way is to monitor the first several iterations and set the learning rate to be higher than the best performing one, but not too high to cause instability.


<p>
    <img src="SGD_lr.png" alt>
</p>

A big advantage of SGD is that the time taken to compute a weight update doesn't grow with number of examples. as each update is computed after observing a batch of samples which is independent of the total number of training examples. Theoretically, for a convex problem BGD makes an error rate of $\mathcal{O}(\frac{1}{k})$ after k iterations whereas SGD makes $\mathcal{O}(\frac{1}{\sqrt{k}})$. However, SGD compensates for this with its advantages after a few iterations along with the ability to make rapid updates in the case of a large training set.

#### Exponentially weighted averages:

<p>
    <img src="ema.png" alt>
</p>

When you plot the data, you end up with the blue scatter plot (noisy).To compute the trends, the local average or a moving average of the temperature, initialize $V_0 = 0$. Then, on every day, average it with a weight of 0.9 times whatever appears as value, plus 0.1 times that day temperature. So, $\theta_1$ here would be the temperature from the first day and on the second day, take a weighted average, 0.9 times the previous value plus 0.1 times today's temperature and so on.The more general formula is V on a given day is 0.9 times V from the previous day, plus 0.1 times the temperature of that day.
    
<p>
    <img src="ema_2.png" alt>
</p>

So, if we compute this and plot it in red, this is what you get. You get a moving average of what's called an exponentially weighted average of the daily temperature. If we take a generic $\beta = 0.9$, then $V_t = \beta V_{t-1} + (1-\beta)\theta_t$. So, $V_t$ can be seen as approximately averaging over $\frac{1}{1-\beta}$ day's temperature. In our case we are averaging over last 10 days temperature, which is the red line. Now if we set $\beta$ close to 1, say 0.98 then we get the green curve and that is basically approximating over 50 days(smoother curve but shifted to the right), adapts more slowly to temperature changes. It is giving a lot of weight to the previous value $V_{t-1}$ than the current temperature. Now if we take the $\beta$ value as 0.5 we get the yellow line, so we are much more susceptible to outliers as we are averaging over just 2 days.

<p>
    <img src="ema_3.png" alt>
</p>

The graph on the upper side represents the data and the one below represents the exponetially decaying function we are multiplying the data with, so $V_{100}$ is basically the sum of the element wise product of the two graphs. The term $\frac{1-\epsilon}{\epsilon}$ has to be equal to $\frac{1}{\exp}$, so based on that we can calculate how many days worth of temperature it is calculating.

#### Gradient Descent with Momentum

It almost always works faster than standard gradient descent algorithm. The basic idea is to compute an exponentially weighted average of your gradients and then use that gradient to update the weights. Let's say we are trying to optimize a cost function with contours like show in the figure.

<p>
    <img src="gradient_descent.gif" alt>
</p>

We start gradient descent from the point that is shown, and we see where we end up as we take steps. As we can observe that gradient descent takes a lot of steps and slowly oscillates towards the minima. This up and down oscillation slows down gradient descent and prevents us from using a very large learning rate. If we use a very high learning rate we might end up overshooting or diverging as shown.

<p>
    <img src="over_div.gif" alt>
</p>

Another way of viewing this problem is that on vertical axis the learning should be slower but on the horizontal axis it has to be faster. 

<p>
    <img src="sgd_3.png" alt>
</p>

If you implement GD with momentum, during iteration t, compute dW, dB on current minibatch and then we compute $V_{dW} = \beta V_{dW} + (1-\beta) dW$ and $V_{dB} = \beta V_{db} + (1-\beta) db$ and then the updates would be $W = W- \alpha V_{dW}$ and $b = b-\alpha V_{db}.$ What it does is it smoothes out the steps of gradient descent. So basically in vertical direction the average becomes closer to zero and in the horizontal direction, they all point to the right so the average in horizontal direction will be pretty big, so it ends up taking steps like shown in red:

<p>
    <img src="mom.png" alt>
</p>

One intuition is that we are trying to minimize a bow shaped function, so the derative terms can be thought of as providing acceleration to a ball that you are rolling downhill and momentum terms are velocity and $\beta$ acts as friction. 


#### RMS Prop



<p>
    <img src="rms_prop.png" alt>
</p>


On, iteration t, we compute dW and db as usual on current minibatch, then we compute $S_{dW}$ and $S_{db}$ as shown in the figure, what it is doing is it is keeping exponential moving averages of the square of the derivatives. Then the update step is such because the gradient in the vertical direction is large so the $S_{db}$ term is large hence the updates are small and in the horizontal direction $S_{dW}$ is small so the updates in horizontal direction are much faster. Here you can then use larget learning rate. 

#### Adam Optimization Algorithm

It takes momentum and RMS prop and puts them together. We initialize V and S to be 0. Then we use momentum to compute V's and S's using RMS prop algorithm. In a typical implementation of Adam we use bias correction as well, which is shown with the corrected parameter. Then the updates are done as shown. 


<p>
    <img src="Adam.png" alt>
</p>

The algorithm has a number of hyperparameters, $\alpha$(learning rate) needs to be tuned, the recommeded value for $\beta_1$ and $\beta_2$ is 0.9 and 0.999. Adam stands for Adaptive Moment Estimation. 

### Parameter Initialization Strategies:

Training algorithms for deep learning are iterative in nature and require specification of an initial point. This is extremely crucuial as it decides whether or not the algorithm converges and if it does, then does the algorithm converge to a point with high cost or low cost.

We have limited understanding of neural network optimization but one thing we know for sure is that neural network should break symmetry. This means that if two hidden units are connected to the same input units, then these should have different initialization or else the gradient would update both the units in the same way and we don't learn anything new by using an additional unit. The idea of having each unit learn something different motivates random initialization of weights which is also computationally cheaper.

Biases are often chosen heuristically (zero mostly) and only the weights are randomly initialized, almost always from a Gaussian or uniform distribution. The scale of the distribution is of utmost concern. Large weights might have better symmetry-breaking effect but might lead to chaos (extreme sensitivity to small perturbations in the input) and exploding values during forward & back propagation. As an example of how large weights might lead to chaos, consider that there’s a slight noise adding $\epsilon$ to the input. Now, we if did just a simple linear transformation like $W * x$, the $\epsilon$ noise would add a factor of $W * \epsilon$ to the output. In case the weights are high, this ends up making a significant contribution to the output. SGD and its variants tend to halt in areas near the initial values, thereby expressing a prior that the path to the final parameters from the initial values is discoverable by steepest descent algorithms.

Various suggestions have been made for appropriate initialization of parameters. The most common ones include sampling of weights of each fully connected layer having m inputs and n outputs uniformly from the following distributions:
   * $U(-\frac{1}{\sqrt{m}},\frac{1}{\sqrt{m}})$
   * $U(-\frac{\sqrt{6}}{m+n},\frac{\sqrt{6}}{m+n})$
   
One drawback of using $\frac{1}{\sqrt{m}}$ as the standard deviation is that the weights end up being small when a layer has too many input/output units. Motivated by the idea to have the total amount of input to each unit independent of the number of input units m, Sparse initialization sets each unit to have exactly k non-zero weights. However, it takes a long time for GD to correct incorrect large values and hence, this initialization might cause problems.

If the weights are too small, the range of activations across the mini-batch will shrink as the activations propagate forward through the network.By repeatedly identifying the ﬁrst layer with unacceptably small activations and increasing its weights, it is possible to eventually obtain a network with reasonable initial activations throughout.

The biases are relatively easier to choose. Setting the biases to zero is compatible with most weight initialization schemes except for a few cases for e.g. when used for an output unit, to prevent saturation at initialization or when using unit as a gate for making a decision.

To motivate all this, let's start by a simple single neuron example, so $z = w_1x_1 + w_2x_2 + w_3x_3 + \ldots w_nx_n$, so to not let z blow up or become too small you notice that the larger n is the smaller you want $w_i$ to be. So one reasonable thing to do would be to set the variance of $w_i$ to be $\frac{1}{n}$, where n is the number of inputs to the network, so in practice $W^{[l]} = np.random.randn(shape)*\sqrt{\frac{1}{n^{[l-1]}}}$ it also turns out that if we use ReLu activation function then $W^{[l]} = np.random.randn(shape)*\sqrt{\frac{2}{n^{[l-1]}}}$ works better. If we use tanh then the former works better. 
